In [1]:
# import files

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
from datetime import date, timedelta
import pandas as pd

import time

In [2]:
# Global variables
search_field_term_job = "data scientist"
search_field_term_location = "New York, NY"


In [3]:
# define driver
driver = webdriver.Chrome()

# open url
driver.get("https://www.simplyhired.com/")

# wait for the page to load
driver.implicitly_wait(5)


# find and click on search field location
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R5bakt9fbqm:"]')
search_field.click()

# clear search field
#search_field.clear()
search_field.send_keys(Keys.CONTROL + "a")
search_field.send_keys(Keys.DELETE)

# enter search term job
search_field.send_keys(search_field_term_location)




# find and click on search field job
search_field = driver.find_element(By.XPATH, '//*[@id="field-:R3bakt9fbqm:"]')
search_field.click()

# wait for the page to load
driver.implicitly_wait(5)

# enter search term job
search_field.send_keys(search_field_term_job)

element = driver.switch_to.active_element
element.send_keys(Keys.ESCAPE)

# wait for the page to load
driver.implicitly_wait(5)



# submit form
# search_field.submit()
search_field.send_keys(Keys.ENTER)


# set for past 24 hours
driver.find_element(By.XPATH, '//*[@id="menu-button-:rg:"]').click()
time.sleep(2)
driver.find_elements(By.XPATH, '//*[@id="menu-list-:rg:"]//button')[1].click()



In [4]:
def pullJobData():
    webElementJobList = driver.find_element(By.XPATH, '//*[@id="job-list"]')
    num_rows = len(webElementJobList.find_elements(By.XPATH, '//li[@class="css-0"]'))
    wait = WebDriverWait(driver, 10)
    

    # if link not found, skip to next job
    skip = False

    # list of jobs
    job_list = []

    for i in range(1, num_rows+1):

        # list in order of appearance company, location, hours, salary, date posted
        # should be 2 or more
        job_desc = {
                    "jobTitle" : '',
                    'company':'',
                    'location':'',
                    'salary':'',
                    'date posted':''
                    }



        # wait for the page to load
        time.sleep(5)

        for tries in range(2):

            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, f'//li[@class="css-0"][{i}]//a')))
                job_desc["jobTitle"] = webElementJobList.find_element(By.XPATH, f'//li[@class="css-0"][{i}]//a').text
                break
            except Exception as e:
                print('job title not found', e)


        for tries in range(2):
            try:
                link = webElementJobList.find_element(By.XPATH, f'//li[@class="css-0"][{i}]//a')\
                    .get_attribute('href')
                break
            except:
                print('link not found')



        # click on job
        time.sleep(3)

        for tries in range(2):
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, f'//li[@class="css-0"][{i}]//a')))
                webElementJobList.find_element(By.XPATH, f'//li[@class="css-0"][{i}]//a').click()
                break
            except:
                print('link not clickable')

                # skip to next job
                skip = True

        if skip:
            continue

        
        time.sleep(3)

        # store values  

        # description
        for tries in range(2):
            xpath = '//*[@data-testid="viewJobBodyJobFullDescriptionContent"]'
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                describtion = driver.find_element(By.XPATH, xpath).text
                break
            except:
                print('description not found')

        


        
        for tries in range(2):
            xpath = '//*[@data-testid="viewJobHeadingContainer"]//span[@data-testid="detailText"]'
            try:
                wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))
                job_desc['company'], job_desc['location']  = [ele.text for ele in driver.\
                       find_elements(By.XPATH, xpath)]
                break
            except:
                print('company and location not found')

            

        for tries in range(2):

            try:

                job_desc['salary']  = driver.find_element(By.XPATH, '//*[@data-testid="viewJobBodyJobCompensation"]').text
                break
            except:
                print('company and location not found')

            
        

        # qualification
        xpath = '//*[@id="__next"]/div/main/div/div[2]/div/div/div[2]/div/aside/div/div[1]/div/div[3]/div/div/ul//li'
        for tries in range(2):
            try:
                qualifications = [detail.text for detail in driver.find_elements(By.XPATH, xpath)]
                qualifications = "||".join(qualifications)
                break
            except:
                print('qualifications not found')




        job_list.append([job_desc['jobTitle'], job_desc['company'],job_desc['location'],job_desc['salary'],\
                    job_desc['date posted'], link, describtion, qualifications ])
    
    return job_list

In [5]:
jobs = []

for _ in range(5):
    jobs.extend(pullJobData())

    try:
        next_page = driver.find_element(By.XPATH, '//*[@id="__next"]//main//nav//a[last()]')
        next_page.click()
    except:
        print('No more pages')
        break

    # let page load(driver.implicitly_wait does not work
    time.sleep(5)




job title not found Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AA532142+3514994]
	(No symbol) [0x00007FF7AA150CE2]
	(No symbol) [0x00007FF7A9FF76AA]
	(No symbol) [0x00007FF7AA041860]
	(No symbol) [0x00007FF7AA04197C]
	(No symbol) [0x00007FF7AA084EE7]
	(No symbol) [0x00007FF7AA06602F]
	(No symbol) [0x00007FF7AA0828F6]
	(No symbol) [0x00007FF7AA065D93]
	(No symbol) [0x00007FF7AA034BDC]
	(No symbol) [0x00007FF7AA035C64]
	GetHandleVerifier [0x00007FF7AA55E16B+3695259]
	GetHandleVerifier [0x00007FF7AA5B6737+4057191]
	GetHandleVerifier [0x00007FF7AA5AE4E3+4023827]
	GetHandleVerifier [0x00007FF7AA2804F9+689705]
	(No symbol) [0x00007FF7AA15C048]
	(No symbol) [0x00007FF7AA158044]
	(No symbol) [0x00007FF7AA1581C9]
	(No symbol) [0x00007FF7AA1488C4]
	BaseThreadInitThunk [0x00007FFA2362257D+29]
	RtlUserThreadStart [0x00007FFA2416AA58+40]

job title not found Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7AA532142+3514994]
	(No symbol) [0x00007FF7AA150CE2]
	(No symbol) [0x00007FF7A9FF7

In [6]:
jobs = pd.DataFrame(jobs, columns = ['jobTitle', 'company', 'location', 'salary', 'date posted', 'link', 'describtion', 'qualifications'])
jobs.head()

,jobTitle,company,location,salary,date posted,link,describtion,qualifications
0,Data Science Lead Analyst (VP) – Data Science,Citi,"New York, NY","$142,320 - $213,480 a year",,https://www.simplyhired.com/job/8BODaQHIRS-YxJ...,Serves as a key member of the Fair Lending Ana...,Data mining||Doctoral degree||SAS||Fair Housin...
1,GHEOR Data Scientist,Intuitive Surgical,"New York, NY","$114,300 - $193,500 a year",,https://www.simplyhired.com/job/dnw6V7dGI3zaEm...,"Company Description\nAt Intuitive, we are unit...",Statistics||SAS||R||HIPAA||Tableau||Master's d...


In [7]:
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   jobTitle        2 non-null      object
 1   company         2 non-null      object
 2   location        2 non-null      object
 3   salary          2 non-null      object
 4   date posted     2 non-null      object
 5   link            2 non-null      object
 6   describtion     2 non-null      object
 7   qualifications  2 non-null      object
dtypes: object(8)
memory usage: 256.0+ bytes


In [8]:
jobs.to_csv(f'simplyHiredScrape_{date.today()}.csv', index = False )

In [9]:
driver.close()